In [1]:
name = '2016-10-07-creating-netcdf-datasets'
title = 'Creating NetCDF datasets'
tags = 'netcdf, io'
author = 'Denis Sergeev'

In [ ]:
from nb_tools import connect_notebook_to_post
from IPython.core.display import HTML

html = connect_notebook_to_post(name, title, tags, author)

### Load essential modules

In [ ]:
import netCDF4 as nc
import numpy as np

In [ ]:
HTML(html)